# Train MRCNN on Septin
##### Braedyn Au 30/8/19

Description

Within the same folder as this notebook (Mask_RCNN), you will find a folder named Images. In there you will see a *test* and *train* folder. Put your respective images into each folder, ensuring a ratio between 80/20 to 90/10 split between training and testing images.

Open Anaconda Powershell Prompt to first use the labelling tool, Labelme. This was installed in a virtual environment as per its developers. First activate this environment

    conda activate labelme
    
You should see the prefix in brackets should change from (base) to (labelme). Then you can run Labelme as a simple command.

    labelme
    
Within the GUI, you must load a directory, either the *test* or *train* folder, and ensure automatic saving is enabled in the options. Using the circle tool (only tool supported currently), draw a circle around each septin ring on each image. These coordinates are saved in a .json file in the same folder as the image. 

After each image has been labelled, you can continue onto the python code below to train the model.

In [1]:
# Import packages
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
from os import listdir
import tensorflow as tf

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


Transfer learning is using prexisting knowledge from other datasets as a starting point to continue training on our data instead of from scratch. Its like instead of teaching a newborn baby how to recognize septin rings, you're teaching a teenager with high school bio knowledge. This is done by loading weights, in this case trained on the COCO dataset which contains real world objects, and adding our objects to the model. 

In [2]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

## Configuration

The default configuration class can be found in the mrcnn folder in the config.py file. However, although most configuration options are fine being left at default, some should be changed for our training. Instead of changing them in the config.py file, we can create a new class based on the original class with updated parameters.

#### Firstly, the STEPS_PER_EPOCH parameter generally should match the number of training samples you have in the train folder.

Learning momentum acts as real world momentum on the gradient descent vector, which can speed up time to reach minimum loss but can also cause hill climb, where the loss reaches a minimum but due to momentum begins to climb the gradient and increase loss.

In [3]:
class SeptinConfig(Config):
    """Configuration for training on the septin dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "Septin_cfg"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + Septin

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 233

    # Skip detections with < 90% confidence not used in training
    # DETECTION_MIN_CONFIDENCE = 0.5

    # Decrease learning momentum
    LEARNING_MOMENTUM = 0.5

## Dataset

Similar to the config class, there is a default dataset class which can be overwritten to match our dataset. Since our labels are created in Labelme, the labels of rings in each image are stored in json files and thus need to be extracted and put into a mask.

In [4]:
class SeptinDataset(Dataset):

    def load_Septin(self, subset):
        """Load a subset of the Septin dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or test
        """
        # Add classes. We have only one class to add.
        self.add_class("Septin", 1, "Septin")

        # Labels are in same folder as images
        assert subset in ["train", "test"]
        dataset_dir =  './Images/'
        dataset_dir = os.path.join(dataset_dir, subset)
        
        for img in listdir(dataset_dir):
            if img.endswith('.tif'):
                label = img[:-4]+'.json'
                if os.path.exists(os.path.join(dataset_dir,label)):
                    # Load annotations
                    annotations = json.load(open(os.path.join(dataset_dir, label)))
                    circles = []
                    polygons = []
                    regions = [a for a in annotations['shapes']]
                    # Take the circle coordinates
                    for a in regions:
                        if a['shape_type'] == 'circle':
                            c,r = a['points'][0]
                            c2,r2 = a['points'][1]
                            radius = np.sqrt((c2-c)*(c2-c)+(r2-r)*(r2-r))
                            circles.append((int(r),int(c),int(radius)))
                        if a['shape_type'] == 'polygon':
                            rpoints = []
                            cpoints = []
                            for p in a['points']:
                                rpoints.append(p[1])
                                cpoints.append(p[0])
                            polygons.append([rpoints,cpoints])
                else:
                    # Empty image as a true negative
                    circles = []
                    polygons = []

                image_path = os.path.join(dataset_dir, annotations['imagePath'])
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

                self.add_image(
                    "Septin",
                    image_id=annotations['imagePath'],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    circles=circles, polygons=polygons)
            else:
                pass

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Septin dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "Septin":
            return super(self.__class__, self).load_mask(image_id)

        # Convert circles to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        maskTemp = np.zeros([info["height"]+200, info["width"]+200, len(info["circles"])+len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["circles"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.circle(p[0]+100, p[1]+100, p[2])
            maskTemp[rr, cc, i] = 1
        for i, p in enumerate(info["polygons"]):
            dimension = i + len(info["circles"])
            rrp, ccp = skimage.draw.polygon(np.add(p[0],100), np.add(p[1],100))
            maskTemp[rrp,ccp, dimension] = 1
        # Crop to include edges
        mask = maskTemp[100:-100,100:-100]

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "Septin":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

## Train
With the config and dataset classes, we can now define a function to train the model. In the final line you will see a model.train() function called. One of the inputs for this is the number of epochs, which means how many times the model looks over all the images. The more epochs the more familiar the model becomes, but too many epochs can lead to overfitting on our training data, which means the model may have a harder time recognizing rings in unseen images. 5-8 epochs is a good range.

You may also notice that the layers input is set to heads, which means we are only training the head layers of the neural network, not the entire network. On the Vutara system, the video ram on the graphics card limits us to this as it is much harder to train the whole network. If more vram is available or if this is implemented on the PowerAnalyse, you can set that input to 'all'.

In [5]:
def train(config=SeptinConfig()):
    """Train the model."""
    # Training dataset.
    dataset_train = SeptinDataset()
    dataset_train.load_Septin("train")
    dataset_train.prepare()
    print('Train: %d' % len(dataset_train.image_ids))

    # Validation dataset
    dataset_val = SeptinDataset()
    dataset_val.load_Septin("test")
    dataset_val.prepare()
    print('Test: %d' % len(dataset_val.image_ids))

    config = SeptinConfig()
    config.display()

    # define the model
    model = MaskRCNN(mode='training', model_dir='./', config=config)
    # load weights (mscoco) and exclude the output layers
    model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
    # train weights (output layers or 'heads')
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=20,
                layers='heads')

### GPU Check
To make sure you are running on the GPU instead of the CPU, run the following line of code. In the Jupyter notebook terminal in your taskbar, you will see which device is being used. If the GPU is configured correctly, something along the likes of 
    
    /job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX...

In [6]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

If the Jupyter command prompt shows GPU, you can continue on to the next cell. If it is CPU, the next step will take so so much longer.

Double check you have the correct parameters, especially STEPS_PER_EPOCH, number of epochs, etc., and you can call the train function to begin.

In [7]:
train()

Train: 233


W0904 14:48:44.303725 13152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 14:48:44.311726 13152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0904 14:48:44.342727 13152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0904 14:48:44.374729 13152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0904 14:48:44.378730 13152 deprecation_wrapper.py:119]

Test: 37

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.5
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE

W0904 14:48:48.108942 13152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0904 14:48:49.082998 13152 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0904 14:48:49.271009 13152 deprecation_wrapper.py:119] From C:\Users\Braedyn Au\Desktop\newmaskrcnn\Mask_RCNN\mrcnn\model.py:553: The name tf.random_shuffle is deprecated. Please use tf.random.shuffle instead.

W0904 14:48:49.365015 13152 deprecation_wrapper.py:119] From C:\Users\Braedyn Au\Desktop\newmaskrcnn\Mask_RCNN\mrcnn\utils.py:202: The name tf.log is deprecat


Starting at epoch 0. LR=0.001

Checkpoint Path: ./septin_cfg20190904T1448\mask_rcnn_septin_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (Tim

W0904 14:48:59.761609 13152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.

Epoch 1/20
233/233 [==============================] - 499s 2s/step - loss: 0.9522 - rpn_class_loss: 0.0586 - rpn_bbox_loss: 0.1628 - mrcnn_class_loss: 0.0707 - mrcnn_bbox_loss: 0.2294 - mrcnn_mask_loss: 0.4307 - val_loss: 0.5782 - val_rpn_class_loss: 0.0186 - val_rpn_bbox_loss: 0.1058 - val_mrcnn_class_loss: 0.0443 - val_mrcnn_bbox_loss: 0.1495 - val_mrcnn_mask_loss: 0.2600
Epoch 2/20
233/233 [==============================] - 496s 2s/step - loss: 0.6160 - rpn_class_loss: 0.0300 - rpn_bbox_loss: 0.1168 - mrcnn_class_loss: 0.0560 - mrcnn_bbox_loss: 0.1494 - mrcnn_mask_loss: 0.2638 - val_loss: 0.4751 - val_rpn_class_loss: 0.0193 - val_rpn_bbox_loss: 0.0936 - val_mrcnn_class_loss: 0.0482 - val_mrcnn_bbox_loss: 0.1083 - val_mrcnn_mask_loss: 0.2058
Epoch 3/20
233/233 [==============================] - 491s 2s/step - loss: 0.5353 - rpn_class_loss: 0.0227 - rpn_bbox_loss: 0.1007 - mrcnn_class_loss: 0.0536 - mrcnn_bbox_loss: 0.1187 - mrcnn_mask_loss: 0.2396 - val_loss: 0.5597 - val_rpn_class_l

RuntimeError: Can't decrement id ref count (unable to extend file properly)